# Infosys Quarterly Report Analysis

#### Developed By: Manaranjan Pradhan
#### www.manaranjanp.com

*This Jupyter notebook is confidential and proprietary to Manaranjan Pradhan. It is intended solely for authorized training purposes. Unauthorized distribution, sharing, or reproduction of this notebook or its contents is strictly prohibited. This material is for personal learning within the training program only and may not be used for commercial purposes or shared with others. Unauthorized use may result in disciplinary action or legal consequences. If you have received this notebook without authorization, please contact manaranjan@gmail.com immediately and delete all copies.*

In [7]:
!pip install getpass4 pypdf openai faiss-cpu llama-index llama-index-readers-file llama-index-vector-stores-faiss llama-index-embeddings-openai llama-index-llms-groq llama-index-embeddings-huggingface

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
import os
from getpass import getpass
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [8]:
from llama_index.core import SimpleDirectoryReader, ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.node_parser import (SentenceWindowNodeParser,)
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core import Document
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [9]:
import faiss

## Configure LLM service

In [10]:
#llm = OpenAI(temperature=0,
#             model="chatgpt-4o-latest",
#             max_tokens=500)


llm = Groq(model="llama-3.3-70b-versatile")

In [11]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

## Load data
Downloaded from

https://www.infosys.com/investors/reports-filings/quarterly-results.html

In [13]:
!pip install docling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.5/166.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.3/142.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 25.2 MB/s eta 0:00:00
   ━━━━━

In [14]:
from docling.document_converter import DocumentConverter

In [15]:
converter = DocumentConverter()
result = converter.convert("/content/ifrs-inr-press-release.pdf")
result.document.export_to_markdown()

# Define the output directory
output_dir = "output_documents"
os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

# Define output file path
output_path = os.path.join(output_dir, "converted_document.md")

# Save as Markdown
with open(output_path, "w", encoding="utf-8") as f:
    f.write(result.document.export_to_markdown())

print(f"Document saved at: {output_path}")


Document saved at: output_documents/converted_document.md


In [16]:
#q1_2024 = SimpleDirectoryReader(
#    input_files=["/content/ifrs-inr-press-release.pdf"]
#).load_data()

q1_2024 = SimpleDirectoryReader(
    input_files=["/content/output_documents/converted_document.md"]
).load_data()

# Build indices

In [17]:
# dimensions of text-ada-embedding-002
d = 384
faiss_index = faiss.IndexFlatL2(d)

In [18]:
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
q1_2024_index = VectorStoreIndex.from_documents(q1_2024, storage_context=storage_context)

## Build query engines

In [19]:
q1_2024__engine = q1_2024_index.as_query_engine(similarity_top_k=2)

In [21]:
# Assuming you have already built the index and query engine
retriever = q1_2024__engine.retriever
retrieved_nodes = retriever.retrieve("What is the revenue growth for the quarter?")

# Print the retrieved chunks
for i, node in enumerate(retrieved_nodes):
    print(f"\n--- Chunk {i+1} ---")
    print(node.get_text())


--- Chunk 1 ---
360                          |
| Basic EPS ( ` )                       | 14.37                          | 12.78                          |
| Diluted EPS ( ` )                     | 14.35                          | 12.76                          |

## IFRS - INR Press Release

<!-- image -->

## IFRS - INR Press Release

## NOTES:

- 1. The above information is extracted from the audited condensed consolidated Balance sheet and Statement of Comprehensive Income for the quarter ended June 30, 2023, which have been taken on record at the Board meeting held on July 20, 2023.
- 2. A Fact Sheet providing the operating metrics of the Company can be downloaded from www.infosys.com.
- 3. Represents bank balance earmarked for final dividend. Payment date for dividend was July 3, 2023.
- 4. Other income is net of Finance Cost.

--- Chunk 2 ---
<!-- image -->

## IFRS - INR  INR Press Release  se IFRS Press Relea

\

Solid Q1 year on year revenue growth of 4.2% at 20.8% operating 

## Run queries

In [ ]:
response = q1_2024__engine.query(
    "What is the revenue growth for the quarter?"
)

In [ ]:
print(response)

The revenue growth for the quarter is 4.2% year on year and 1.0% sequentially in constant currency.


In [ ]:
response = q1_2024__engine.query("Which are some of the key customer wins?")

In [ ]:
print(response)

Some of the key customer wins include XacBank, for which Infosys Finacle helped transform its technology landscape with the Finacle Digital Banking Suite, enabling a robust digital foundation for the bank to achieve its growth strategy.


In [ ]:
response = q1_2024__engine.query("What are some of the key achievements in the quarter?")

In [ ]:
print(response)

Some of the key achievements in the quarter include a 4.2% year-on-year revenue growth, 20.8% operating margins, a 12.4% year-on-year increase in EPS, and a large deal TCV of $2.3 billion. Additionally, the company reported a 10.0% year-on-year growth in reported revenues and a 12.6% year-on-year growth in free cash flow, with a free cash flow conversion of 96.7% of net profit.


In [ ]:
response = q1_2024__engine.query("What is the total assets?")

In [ ]:
print(response)

The total liabilities and equity is 131,322. Since assets = liabilities + equity, the total assets is also 131,322.
